# 构建一个代理

使用LangSmith构建应用程序，允许检查链和代理内部发生什么。将API 密钥存入了系统环境变量中。LangChain 框架会自动从环境变量中读取这个密钥，无需在后续调用组件（如链、代理、模型等）时重复传入。

In [2]:
!pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

     ---------------------------------------- 0.0/44.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/44.7 kB ? eta 

In [5]:
import getpass
import os

In [8]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


我们将使用 Tavily（一个搜索引擎）作为工具。 为了使用它，您需要获取并设置一个API密钥：

In [10]:
os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


### 定义工具

我们首先需要创建我们想要使用的工具。我们主要选择的工具将是 Tavily - 一个搜索引擎。LangChain中内置了一个工具，可以轻松使用Tavily搜索引擎作为工具。

In [16]:
!pip install -U langchain-tavily

     ---------------------------------------- 0.0/76.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/76.3 kB ? eta -:--:--
     --------------- ---------------------- 30.7/76.3 kB 435.7 kB/s eta 0:00:01
     -------------------- ----------------- 41.0/76.3 kB 330.3 kB/s eta 0:00:01
     -------------------- ----------------- 41.0/76.3 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 76.3/76.3 kB 423.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/450.3 kB ? eta -:--:--
   -- ------------------------------------- 30.7/450.3 kB 1.3 MB/s eta 0:00:01
   ------- -------------------------------- 81.9/450.3 kB 1.5 MB/s eta 0:00:01
   ---------- ----------------------------- 122.9/450.3 kB 1.2 MB/s eta 0:00:01
   -------------- ------------------------- 163.8/450.3 kB 1.1 MB/s eta 0:00:01
   ---------------------- ----------------- 256.0/450.3 kB 1.3 MB/s eta 0:00:01
   ---------------------------------- ----- 389.1/450.3 kB 1.6 MB/s

In [20]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("what is the weather in Shanghai")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'what is the weather in Shanghai', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in Shanghai', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Shanghai', 'region': 'Shanghai', 'country': 'China', 'lat': 31.005, 'lon': 121.4086, 'tz_id': 'Asia/Shanghai', 'localtime_epoch': 1755052156, 'localtime': '2025-08-13 10:29'}, 'current': {'last_updated_epoch': 1755051300, 'last_updated': '2025-08-13 10:15', 'temp_c': 34.2, 'temp_f': 93.6, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 13.6, 'wind_kph': 22.0, 'wind_degree': 164, 'wind_dir': 'SSE', 'pressure_mb': 1011.0, 'pressure_in': 29.85, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 53, 'cloud': 0, 'feelslike_c': 45.8, 'feelslike_f': 114.5, 'windchill_c': 31.0, 'windchill_f': 87.7, 'heatindex_c': 36.8, 'heatindex_f': 98.2, 'dewpoint_c': 24.3, 'dewpoint_f': 75.8, 'vis_km': 10.0, 'vis

### 使用语言模型

In [23]:
from langchain_openai import ChatOpenAI

In [25]:
api_key = getpass.getpass("DeepSeek API key: ")

DeepSeek API key:  ········


In [29]:
model = ChatOpenAI(
    model="deepseek-chat",
    temperature=0.3,
    max_tokens=200,
    api_key=api_key, # Deepseek api key
    base_url="https://api.deepseek.com/v1"
)

In [31]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! 😊 How can I help you today?'

将模型和工具绑定，让语言模型了解这些工具：

通过这种绑定，模型能够根据输入判断是否需要调用工具，从而具备工具调用的能力，后续可以观察到模型的工具调用情况。

In [36]:
model_with_tools = model.bind_tools(tools)

现在可以调用模型。让我们先用一条普通消息调用它，看看它的响应。我们可以查看 content 字段和 tool_calls 字段。

In [39]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today? 😊
ToolCalls: []


现在，让我们尝试用一些输入来调用它，这些输入会期望调用一个工具。

In [42]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search', 'args': {'query': 'current weather in San Francisco', 'search_depth': 'basic'}, 'id': 'call_0_4ea3ee23-0e1e-4646-a453-39f335f849ca', 'type': 'tool_call'}]


### 创建代理

使用LangGraph来构建代理，使用高级接口构建代理。LangGraph是LangChain生态系统中的一个专门库，采用图结构来表示任务和流程，节点代表操作或步骤，边代表节点之间的依赖关系，它能构建更多复杂系统、多智能体系统和非线性工作流程。

1. 现在，我们可以用大型语言模型和工具初始化代理。

请注意，我们传入的是 model，而不是 model_with_tools。这是因为 create_react_agent 会在后台为我们调用 .bind_tools。

In [48]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

### 运行代理

尝试在几个查询上运行代理（此处查询指向代理提出的问题或请求，如“hi”、“查询天气”），目前使用的都是**无状态查询**，即不记住之前历史交互。代理将在交互结束时返回**最终状态**，即代理在处理完一个查询请求后，所形成的包含此次交互的完整记录（用户输入内容、代理生成的响应、调用工具获取的信息等）

2. 现在先对一个不会调用工具的查询运行代理

In [61]:
response = agent_executor.invoke({
    "messages": [HumanMessage(content="hi!")]
})
response["messages"] # agent_executor.invoke(...)返回的是一个包含完整交互流程的字典（而非单一消息对象）。此为代理返回的最终状态

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='d7a4dee6-2ada-4dfa-a33a-72cd454ec65a'),
 AIMessage(content='Hello! How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 1944, 'total_tokens': 1955, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1920}, 'prompt_cache_hit_tokens': 1920, 'prompt_cache_miss_tokens': 24}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': '80b289ef-6006-482c-a9db-be02db2a19aa', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--eea172e9-8ffb-419f-893e-a4f7d4feb69c-0', usage_metadata={'input_tokens': 1944, 'output_tokens': 11, 'total_tokens': 1955, 'input_token_details': {'cache_read': 1920}, 'output_token_details': {}})]

可以查看LangSmith了解后台发生了什么，网址：https://smith.langchain.com/o/289d41e9-e8de-4306-ab3c-0ea35f3db4f2/projects/p/14498a88-9a17-4068-9ced-e6a06b90fb8e?timeModel=%7B%22duration%22%3A%227d%22%7D

3. 再试一个会调用工具的示例运行代理。

In [67]:
response = agent_executor.invoke({
    "messages": [HumanMessage(content="What's the weather in SF?")]
})
response["messages"]

[HumanMessage(content="What's the weather in SF?", additional_kwargs={}, response_metadata={}, id='d5cd8f96-79d0-4e81-b960-77d0fd80807f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_60b777a3-a47e-431f-aa52-4517d81f34b0', 'function': {'arguments': '{"query":"current weather in San Francisco","search_depth":"basic"}', 'name': 'tavily_search'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 1949, 'total_tokens': 1978, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1920}, 'prompt_cache_hit_tokens': 1920, 'prompt_cache_miss_tokens': 29}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': '7fdf2921-4cf4-4abe-bd47-25730f3afdc1', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a8bcd491-acc0-457a-97ab-180808dd2f4d-0', tool_calls=[{'name': 'tavily_search', 

由输出可以看到返回中有ToolMessage信息，说明调用了工具。可以再查看LangSmith，确保后台调用了工具。

### 流式消息

如果代理正在执行多个步骤，这可能需要一些时间。为了显示中间进度，我们可以在消息发生时流式返回消息。

In [72]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_52795776-90fa-417e-9926-e44a7df69e8f', 'function': {'arguments': '{"query":"current weather in San Francisco","search_depth":"basic"}', 'name': 'tavily_search'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 1949, 'total_tokens': 1978, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1920}, 'prompt_cache_hit_tokens': 1920, 'prompt_cache_miss_tokens': 29}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': '1d39274f-cacd-4446-bc48-abb6b1d28c4e', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--bed75ce1-8e88-4073-8dea-d79bcf5581ed-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'current weather in San Francisco', 'search_depth': 'basic'}, 'id': 'call_0_52795776-90fa-417e-992

### 流式令牌

除了流式返回消息，流式返回令牌也是有用的。 我们可以使用 .astream_events 方法来实现这一点。

解释下面代码：

这段代码使用agent_executor.astream_events方法异步流式获取代理处理查询（“whats the weather in sf?”）过程中的各类事件。通过判断事件类型（kind）执行不同操作：
* 当事件为on_chain_start且名称为 “Agent” 时，打印代理开始信息及输入；
* 事件为on_chain_end且名称为 “Agent” 时，打印代理结束信息及输出；
* 事件为on_chat_model_stream时，若有内容则打印模型流式输出的令牌；
* 事件为on_tool_start时，打印工具开始调用的信息及输入；
* 事件为on_tool_end时，打印工具调用完成的信息及输出结果。

含义解释：

*异步生成*：指astream_events方法在生成事件（如工具调用、模型输出片段）时，不会一次性阻塞等待所有事件生成完毕，而是一边生成、一边 “推送” 事件，期间允许程序做其他事（非阻塞）。

*流式输出*：指事件 / 结果不是一次性返回，而是分片段、按顺序逐步输出（如模型的 token 一个接一个产生）。

In [78]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search with inputs: {'query': 'current weather in San Francisco', 'search_depth': 'basic'}
Done tool: tavily_search
Tool output was: content='{"query": "current weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1755067374, \'localtime\': \'2025-08-12 23:42\'}, \'current\': {\'last_updated_epoch\': 1755066600, \'last_updated\': \'2025-08-12 23:30\', \'temp_c\': 16.1, \'temp_f\': 61.0, \'is_day\': 0, \'condition\': {\'text\': \'Overcast\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/122.png\', \'code\': 1009}, \'wind_mph\': 9.4, \'wind_kph\': 15.1, \'wind_degree\': 252, \'wind_dir\': \'WSW\', \'pressure_mb\': 1015.

### 添加内存

现在，若想要代理是有状态的，即记住历史交互，则需要传入一个检查点来添加内存。

**检查点**：类似于带结构的“缓存”，存储结构化的状态记录（历史消息）。

**thread_id**：当代理同时处理多个独立任务（比如多个用户的对话、多个并行的工作流）时，每个任务对应一个唯一的 thread_id。调用代理时传入 thread_id，就像在检查点中 “查字典”，通过这个键精准找到该任务对应的历史状态，避免不同任务的上下文混淆。

**配合逻辑**：调用代理时，通过 thread_id 从检查点中加载对应任务的历史交互信息，代理结合这些上下文理解当前任务（比如用户当前提问与历史对话的关联），处理完成后再将新的状态更新到检查点中，供下一次调用使用。

In [86]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [88]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [90]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob! How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 1946, 'total_tokens': 1958, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1920}, 'prompt_cache_hit_tokens': 1920, 'prompt_cache_miss_tokens': 26}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': 'dc336536-d723-446b-8e4a-321db4e5baf4', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--726a8acc-919a-4318-b756-5d35f7fe1a92-0', usage_metadata={'input_tokens': 1946, 'output_tokens': 12, 'total_tokens': 1958, 'input_token_details': {'cache_read': 1920}, 'output_token_details': {}})]}}
----


In [92]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob! 😊 How can I help you, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 1966, 'total_tokens': 1981, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1920}, 'prompt_cache_hit_tokens': 1920, 'prompt_cache_miss_tokens': 46}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': 'd9634448-5813-4d81-ba9f-937c588251a6', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--db9a8a36-d67f-4b6b-a6be-a32860b03f3a-0', usage_metadata={'input_tokens': 1966, 'output_tokens': 15, 'total_tokens': 1981, 'input_token_details': {'cache_read': 1920}, 'output_token_details': {}})]}}
----


可查看LangSmith查看后台追踪。

如果我想开始一个新的对话，我所要做的就是更改使用的 thread_id

In [97]:
config = {"configurable": {"thread_id": "abc120"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don’t have access to personal information about you, including your name. You can tell me your name if you'd like, and I can refer to you that way! 😊", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 1947, 'total_tokens': 1985, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1920}, 'prompt_cache_hit_tokens': 1920, 'prompt_cache_miss_tokens': 27}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': '516c2956-a13e-4d51-8128-2513325fd0fe', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--0d7da314-8f30-4772-a60f-cc91b6fb608e-0', usage_metadata={'input_tokens': 1947, 'output_tokens': 38, 'total_tokens': 1985, 'input_token_details': {'cache_read': 1920}, 'output_token_details': {}})]}}
----


example 3: 再测试代理调用工具时是否查看了历史状态

In [102]:
config = {"configurable": {"thread_id": "abc121"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="I'm living in Shanghai now.")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="That sounds exciting! Shanghai is a vibrant and dynamic city with a mix of modern skyscrapers, rich history, and diverse culture. Whether you're there for work, study, or just exploring, there's always something new to discover.\n\nHere are a few things you might find interesting or useful about living in Shanghai:\n\n1. **Food**: Shanghai is famous for its delicious local cuisine, like xiaolongbao (soup dumplings), shengjianbao (pan-fried buns), and hairy crab (in season). There are also plenty of international dining options.\n\n2. **Transportation**: The city has an extensive and efficient metro system, making it easy to get around. Taxis and ride-hailing apps like Didi are also widely used.\n\n3. **Culture**: Explore places like the Bund, Yu Garden, and the French Concession for a mix of history and modernity. The city also has a thriving arts and music scene.\n\n4. **Shopping**: From luxury brands on Nanjing Road to quirky boutiques in Ti

In [104]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in the place where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_afdb975d-aa6d-444b-8a49-4413982429cf', 'function': {'arguments': '{"query":"current weather in Shanghai","include_domains":["weather.com","accuweather.com","bbc.com/weather"]}', 'name': 'tavily_search'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 2251, 'total_tokens': 2294, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 2176}, 'prompt_cache_hit_tokens': 2176, 'prompt_cache_miss_tokens': 75}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': '2f1efe0b-eede-470a-862a-09fa9a257b7f', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--fbf6d66b-c38d-40f2-a826-17b0f649bfa1-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'current weather in Shanghai', 'include_domains': ['weat

由输出结果以及LangSmith的后台追踪，可以看出确实调用工具前查看了历史状态。

### 总结

* 配置LangSmith密钥，允许后台追踪调用记录。配置Tavily搜索引擎密钥，允许后续代理调用该工具获取信息。
* 使用LangChain方法将模型和工具绑定，让模型判断是否要调用工具。
* 使用LangGraph创建代理，实现无状态查询，即不记录历史消息地执行任务。
* 使用流式消息和流式令牌，进行流式输出返回消息
* 传入检查点来添加内存，实现有状态查询，即执行任务时会先调取历史记录，理解上下文，执行后更新记录存入内存。